In [1]:
# Dependencies
import matplotlib.pyplot as plt
import requests
import pandas as pd
from pprint import pprint
from datetime import datetime
import os
import csv
# from config import api_key

In [2]:
# Save config information.
url = "https://data.ca.gov/api/3/action/datastore_search?resource_id=d73ee828-c2c6-485c-91dc-c26bd9ce3991&"

# Build partial query URL
limit = 1000
year = 2020
query_url = f"{url}&limit={limit}&q={year}"
response = requests.get(query_url).json()
pprint(response)

{'help': 'https://data.ca.gov/api/3/action/help_show?name=datastore_search',
 'result': {'_links': {'next': '/api/3/action/datastore_search?limit=1000&q=2020&resource_id=d73ee828-c2c6-485c-91dc-c26bd9ce3991&offset=1000',
                       'start': '/api/3/action/datastore_search?limit=1000&q=2020&resource_id=d73ee828-c2c6-485c-91dc-c26bd9ce3991'},
            'fields': [{'id': '_id', 'type': 'int'},
                       {'id': 'OBJECTID', 'type': 'text'},
                       {'id': 'Year', 'type': 'text'},
                       {'id': 'State', 'type': 'text'},
                       {'id': 'Agency', 'type': 'text'},
                       {'id': 'Unit ID', 'type': 'text'},
                       {'id': 'Fire Name', 'type': 'text'},
                       {'id': 'Local Incident Number', 'type': 'text'},
                       {'id': 'Alarm Date', 'type': 'text'},
                       {'id': 'Containment Date', 'type': 'text'},
                       {'id': 'Cause', 'type': 

In [3]:
df = pd.DataFrame(response['result']['records'])
df_2020 = df.loc[df['Year'] == f'{str(year)}']
columns_to_keep = ['Year', 'Alarm Date','Containment Date','Cause','GIS Calculated Acres']
columns_to_drop = [col for col in df_2020.columns if col not in columns_to_keep]
df_2020 = df_2020.drop(columns=columns_to_drop)
df_2020 = df_2020.dropna(subset=['Containment Date'])
df_2020.head()

,Year,Alarm Date,Containment Date,Cause,GIS Calculated Acres
0,2020,6/30/2020 12:00:00 AM,6/30/2020 12:00:00 AM,14,59.73152
1,2020,11/6/2020 12:00:00 AM,11/6/2020 12:00:00 AM,7,128.1708
2,2020,6/18/2020 12:00:00 AM,6/23/2020 12:00:00 AM,11,109.6023
3,2020,6/1/2020 12:00:00 AM,6/4/2020 12:00:00 AM,2,685.585
4,2020,8/10/2020 12:00:00 AM,8/11/2020 12:00:00 AM,14,27.30048


In [4]:
alarm_date_20 = []
containment_date_20 = []
acres_20 = []
for index, row in df_2020.iterrows():
    alarm_date_20.append(row['Alarm Date'])
    containment_date_20.append(row['Containment Date'])
    acres_20.append(row['GIS Calculated Acres'])
alarm_date_20 = [dt.replace(' 12:00:00 AM', '') for dt in alarm_date_20]
containment_date_20 = [str(item) for item in containment_date_20]
containment_date_20 = [dt.replace(' 12:00:00 AM', '') for dt in containment_date_20]

In [5]:
df_2020['Alarm Date'] = [dt.replace(' 12:00:00 AM', '') for dt in df_2020['Alarm Date']]
df_2020['Containment Date'] = [str(item) for item in df_2020['Containment Date']]
df_2020['Containment Date'] = [dt.replace(' 12:00:00 AM', '') for dt in df_2020['Containment Date']]
df_2020.head()

,Year,Alarm Date,Containment Date,Cause,GIS Calculated Acres
0,2020,6/30/2020,6/30/2020,14,59.73152
1,2020,11/6/2020,11/6/2020,7,128.1708
2,2020,6/18/2020,6/23/2020,11,109.6023
3,2020,6/1/2020,6/4/2020,2,685.585
4,2020,8/10/2020,8/11/2020,14,27.30048


In [6]:
df_2020 = df_2020.rename(columns={'Cause': 'Cause #'})
df_2020['Cause'] = ''
df_2020['Cause #'] = df_2020['Cause #'].astype(int)
def map_cause_to_meaning(i):
    if i == 1:
        return 'Lightning'
    elif i == 2:
        return 'Equipment Use'
    elif i == 3:
        return 'Smoking'
    elif i == 4:
        return 'Campfire'
    elif i == 5:
        return 'Debris'
    elif i == 6:
        return 'Railroad'
    elif i == 7:
        return 'Arson'
    elif i == 8:
        return 'Playing with fire'
    elif i == 9:
        return 'Miscellaneous'
    elif i == 10:
        return 'Vehicle'
    elif i == 11:
        return 'Powerline'
    elif i == 12:
        return 'Firefighter Training'
    elif i == 13:
        return 'Non-Firefighter Training'
    elif i == 14:
        return 'Unknown / Unidentified'
    elif i == 15:
        return 'Structure'
    elif i == 16:
        return 'Aircraft'
    elif i == 18:
        return 'Escaped Prescribed Burn'
    elif i == 19:
        return 'Illegal Alien Campfire'
    else:
        return 'N/A'
df_2020['Cause'] = df_2020['Cause #'].apply(map_cause_to_meaning)
df_2020.head()

,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
0,2020,6/30/2020,6/30/2020,14,59.73152,Unknown / Unidentified
1,2020,11/6/2020,11/6/2020,7,128.1708,Arson
2,2020,6/18/2020,6/23/2020,11,109.6023,Powerline
3,2020,6/1/2020,6/4/2020,2,685.585,Equipment Use
4,2020,8/10/2020,8/11/2020,14,27.30048,Unknown / Unidentified


In [7]:
df_2020.to_csv("outputs/fires_2020.csv",
                  encoding="utf-8", index=False, header=True)

In [8]:
df_2020['Alarm Date'] = pd.to_datetime(df_2020['Alarm Date'])
df_byalarm = df_2020.sort_values(by='Alarm Date').reset_index()
df_byalarm.head()

,index,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
0,468,2020,2020-02-12,2/12/2020,14,2.159534,Unknown / Unidentified
1,30,2020,2020-02-15,2/15/2020,5,10.47374,Debris
2,300,2020,2020-02-17,2/18/2020,5,101.7666,Debris
3,350,2020,2020-02-17,2/24/2020,9,13.9922,Miscellaneous
4,33,2020,2020-02-22,2/23/2020,18,181.4404,Escaped Prescribed Burn


In [9]:
earliest_fire = df_byalarm['Alarm Date'][0]
earliest_fire

Timestamp('2020-02-12 00:00:00')

In [10]:
df_2020['Containment Date'] = pd.to_datetime(df_2020['Containment Date'])
df_bycontainment = df_2020.sort_values(by='Containment Date', ascending = False).reset_index()
latest_containment = df_bycontainment['Containment Date'][0]
df_bycontainment.head()

,index,Year,Alarm Date,Containment Date,Cause #,GIS Calculated Acres,Cause
0,359,2020,2021-12-30,2021-12-31,14,3052.173,Unknown / Unidentified
1,131,2020,2020-10-25,2021-01-25,7,133.2995,Arson
2,134,2020,2020-09-14,2021-01-14,10,17.10545,Vehicle
3,237,2020,2020-08-19,2021-01-04,1,840.8506,Lightning
4,239,2020,2020-08-19,2021-01-04,1,170647.9,Lightning


In [11]:
latest_containment

Timestamp('2021-12-31 00:00:00')

In [12]:
output_path = os.path.join("outputs", "timeline_dates_2020.csv")
with open(output_path, 'w') as csvfile:

    # Initialize csv.writer
    csvwriter = csv.writer(csvfile, delimiter=',')

    # Write the first row (column headers)
    csvwriter.writerow(['Title', 'Date'])
    csvwriter.writerow(['Earliest Fire', earliest_fire])

    # Write the second row
    csvwriter.writerow(['Latest Containment', latest_containment])